In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
from skimage.io import imread 
from PIL import Image
import csv
import torch
import copy
import torchvision
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
from torch.utils import data
import torchvision.transforms as transf
import torch.nn.functional as F
from torchvision.transforms.transforms import RandomAdjustSharpness
import torch.nn as nn
import random
import cv2
import math
import os
from torch import optim
from torch.nn.modules.conv import ConvTranspose2d
from torch.autograd import Function
import glob

## Our dataset's path and preprocessing


Structure of dataset should be as following:

--- /input
------/image1.png
------/image2.png
------/image3.png  
...

--- /target
------/mask1.png
------/mask2.png
------/mask3.png  
...

 Dataset pipeline 
--- ordered list of corresponding images and mask paths --- 

img_list --> ['image1.png', 'image2.png', 'image3.png',...]

msk_list --> ['mask1.png', 'mask2.png', 'mask3.png',...]

In [ ]:
class SegDataset(data.Dataset):
  
  def __init__(self, img_list, msk_list, transform=None):
    self.img_list = img_list
    self.msk_list = msk_list
    self.transforms = transform

  def __len__(self):
    return len(self.img_list)

  @classmethod
  def preprocess(self,pil_img, size):
    # w,h = pil_img.size
    # W,H = int(scale*w), int(scale*h)
    pil_img = pil_img.resize((size,size))

    img_nd = np.array(pil_img)

    if len(img_nd.shape) == 2:
      img_nd = np.expand_dims(img_nd, axis=2)

    '''if tranforms are present paste them here'''

    #HWC to CHW
    img_trans = img_nd.transpose((2,0,1))
    if img_trans.max() > 1:
      img_trans = img_trans/225

    return img_trans

  def __getitem__(self, index):
    input_ID = self.img_list[index]
    target_ID = self.msk_list[index]

    img = Image.open(input_ID)
    msk = Image.open(target_ID)

    ''' 
    assert img.size == msk.size, \
    f'Image and mask for {index} should be the same, but are {img.size} and {msk.size}'

    # preprocessing
    img = self.preprocess(img, 224)
    msk = self.preprocess(msk, 216)
    '''
    return {
        'image': torch.from_numpy(img).type(torch.FloatTensor),
        'mask': torch.from_numpy(msk).type(torch.FloatTensor)
    }

### Hyperparameter Tuning and Dataloader

In [ ]:
# Hyperparameter Tuning
batch_size_tr = 1
batch_size_val = 1
epoch_n = 200
n_channels = 3
n_classes = 1
l_rate = 0.01
'''
img_scale = 1
mask_threshold = 0.5
'''

In [ ]:
train_ds = SegDataset(input_tr, target_tr)#,transform=transforms_tr)
val_ds = SegDataset(input_val, target_val)#,transform=transforms_val)
# test_ds = ImageFolder('/content/drive/MyDrive/Lung_Carcinoma/data_folder_2/test/', transform=val_transform)
train_load = DataLoader(dataset=train_ds, batch_size=batch_size_tr, shuffle=False, drop_last=False)
val_load = DataLoader(dataset=val_ds, batch_size=batch_size_val, shuffle=False, drop_last=False)
# test_load = DataLoader(dataset=test_ds, batch_size=batch_size_val, shuffle=True, drop_last=False)
if torch.cuda.is_available():
  device='cuda'
else:
  device='cpu'

### Structure of U-Net

In [ ]:
# Segmentation architecture
# U-Net

class DoubleConv(nn.Module):
  def __init__(self, in_channels, out_channels, mid_channels=None):
    super().__init__()
    if mid_channels is None:
      mid_channels = out_channels
    self.double_conv = nn.Sequential(
        nn.Conv2d(in_channels, mid_channels, 3, 1),
        nn.BatchNorm2d(mid_channels),
        nn.ReLU(inplace=True),
        nn.Conv2d(mid_channels, out_channels, 3, 1),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(inplace=True)
    )

  def forward(self, x):
    return self.double_conv(x)


class Down(nn.Module):
  def __init__(self, in_channels, out_channels):
    super().__init__()
    self.maxpool_conv = nn.Sequential(
        nn.MaxPool2d(2),
        DoubleConv(in_channels, out_channels)
    )

  def forward(self, x):
    return self.maxpool_conv(x)


class Up(nn.Module):
  def __init__(self, in_channels, out_channels):
    super().__init__()
    self.up = nn.ConvTranspose2d(in_channels, in_channels//2, 2, 2)
    self.conv = DoubleConv(in_channels, out_channels)

  def forward(self, x1, x2):
    x1=self.up(x1)
    delta_W = x2.size()[3]-x1.size()[3]
    delta_H = x2.size()[2]-x1.size()[2]
    x1 = F.pad(x1,[delta_W//2,delta_W-delta_W//2,delta_H//2,delta_H-delta_H//2])
    x = torch.cat([x2,x1],dim=1)
    return self.conv(x)


class FinalConv(nn.Module):
  def __init__(self, in_channels, out_channels):
    super().__init__()
    self.conv_final = nn.Conv2d(in_channels, out_channels,1)

  def forward(self,x):
    return self.conv_final(x)


class U_Net(nn.Module):
  def __init__(self, in_channels, out_classes):
    super(U_Net,self).__init__()
    self.in_channels = in_channels
    self.out_classes = out_classes

    self.initial_conv = DoubleConv(in_channels, 64)
    self.down1 = Down(64, 128)
    self.down2 = Down(128, 256)
    self.down3 = Down(256, 512)
    self.down4 = Down(512, 1024)
    self.up1 = Up(1024,512)
    self.up2 = Up(512,256)
    self.up3 = Up(256,128)
    self.up4 = Up(128,64)
    self.final = FinalConv(64, out_classes)
    
  
  def forward(self,i):
    i1 = self.initial_conv(i)
    i2 = self.down1(i1)
    i3 = self.down2(i2)
    i4 = self.down3(i3)
    i5 = self.down4(i4)
    i = self.up1(i5,i4)
    i = self.up2(i,i3)
    i = self.up3(i,i2)
    i = self.up4(i,i1)
    i = self.final(i)
    return i

### Loss Functions (Dice Coeff and IOU Score)

In [ ]:
# loss functions

'''Dice coeff'''

class DiceCoeff(Function):
    """Dice coeff for individual examples"""

    def forward(self, input, target):
        self.save_for_backward(input, target)
        eps = 0.0001
        self.inter = (input * target).sum(dim=(1, 2))
        self.union = (input + target).sum(dim=(1, 2)) + eps -self.inter

        t = (2 * self.inter.float() + eps) / self.union.float()
        return t.mean()

    # This function has only a single output, so it gets only one gradient
    def backward(self, grad_output):

        input, target = self.saved_variables
        grad_input = grad_target = None

        if self.needs_input_grad[0]:
            grad_input = grad_output * 2 * (target * self.union - self.inter) \
                         / (self.union * self.union)
        if self.needs_input_grad[1]:
            grad_target = None

        return grad_input, grad_target


def dice_coeff(input, target):
    """Dice coeff for batches"""
    if input.is_cuda:
        s = torch.FloatTensor(1).cuda().zero_()
    else:
        s = torch.FloatTensor(1).zero_()

    for i, c in enumerate(zip(input, target)):
        s = s + DiceCoeff().forward(c[0], c[1])

    return s 

### Evaluation metrics ###
class Metric(object):
    """Base class for all metrics.
    From: https://github.com/pytorch/tnt/blob/master/torchnet/meter/meter.py
    """
    def reset(self):
        pass

    def add(self):
        pass

    def value(self):
        pass

'''Confusion Matrix'''

class ConfusionMatrix(Metric):
    """Constructs a confusion matrix for a multi-class classification problems.
    Does not support multi-label, multi-class problems.
    Keyword arguments:
    - num_classes (int): number of classes in the classification problem.
    - normalized (boolean, optional): Determines whether or not the confusion
    matrix is normalized or not. Default: False.
    Modified from: https://github.com/pytorch/tnt/blob/master/torchnet/meter/confusionmeter.py
    """

    def __init__(self, num_classes, normalized=False):
        super().__init__()

        self.conf = np.ndarray((num_classes, num_classes), dtype=np.int64)
        self.normalized = normalized
        self.num_classes = num_classes
        self.reset()

    def reset(self):
        self.conf.fill(0)

    def add(self, predicted, target):
        """Computes the confusion matrix
        The shape of the confusion matrix is K x K, where K is the number
        of classes.
        Keyword arguments:
        - predicted (Tensor or numpy.ndarray): Can be an N x K tensor/array of
        predicted scores obtained from the model for N examples and K classes,
        or an N-tensor/array of integer values between 0 and K-1.
        - target (Tensor or numpy.ndarray): Can be an N x K tensor/array of
        ground-truth classes for N examples and K classes, or an N-tensor/array
        of integer values between 0 and K-1.
        """
        # If target and/or predicted are tensors, convert them to numpy arrays
        if torch.is_tensor(predicted):
            predicted = predicted.cpu().numpy()
        if torch.is_tensor(target):
            target = target.cpu().numpy()

        assert predicted.shape[0] == target.shape[0], \
            'number of targets and predicted outputs do not match'

        if np.ndim(predicted) != 1:
            assert predicted.shape[1] == self.num_classes, \
                'number of predictions does not match size of confusion matrix'
            predicted = np.argmax(predicted, 1)
        else:
            assert (predicted.max() < self.num_classes) and (predicted.min() >= 0), \
                'predicted values are not between 0 and k-1'

        if np.ndim(target) != 1:
            assert target.shape[1] == self.num_classes, \
                'Onehot target does not match size of confusion matrix'
            assert (target >= 0).all() and (target <= 1).all(), \
                'in one-hot encoding, target values should be 0 or 1'
            assert (target.sum(1) == 1).all(), \
                'multi-label setting is not supported'
            target = np.argmax(target, 1)
        else:
            assert (target.max() < self.num_classes) and (target.min() >= 0), \
                'target values are not between 0 and k-1'

        # hack for bincounting 2 arrays together
        x = predicted + self.num_classes * target
        bincount_2d = np.bincount(
            x.astype(np.int64), minlength=self.num_classes**2)
        assert bincount_2d.size == self.num_classes**2
        conf = bincount_2d.reshape((self.num_classes, self.num_classes))

        self.conf += conf

    def value(self):
        """
        Returns:
            Confustion matrix of K rows and K columns, where rows corresponds
            to ground-truth targets and columns corresponds to predicted
            targets.
        """
        if self.normalized:
            conf = self.conf.astype(np.float32)
            return conf / conf.sum(1).clip(min=1e-12)[:, None]
        else:
            return self.conf



'''IoU'''

class IoU(Metric):
    """Computes the intersection over union (IoU) per class and corresponding
    mean (mIoU).
    Intersection over union (IoU) is a common evaluation metric for semantic
    segmentation. The predictions are first accumulated in a confusion matrix
    and the IoU is computed from it as follows:
        IoU = true_positive / (true_positive + false_positive + false_negative).
    Keyword arguments:
    - num_classes (int): number of classes in the classification problem
    - normalized (boolean, optional): Determines whether or not the confusion
    matrix is normalized or not. Default: False.
    - ignore_index (int or iterable, optional): Index of the classes to ignore
    when computing the IoU. Can be an int, or any iterable of ints.
    """

    def __init__(self, num_classes, normalized=False, ignore_index=None):
        super().__init__()
        self.conf_metric = ConfusionMatrix(num_classes, normalized)

        if ignore_index is None:
            self.ignore_index = None
        elif isinstance(ignore_index, int):
            self.ignore_index = (ignore_index,)
        else:
            try:
                self.ignore_index = tuple(ignore_index)
            except TypeError:
                raise ValueError("'ignore_index' must be an int or iterable")

    def reset(self):
        self.conf_metric.reset()

    def add(self, predicted, target):
        """Adds the predicted and target pair to the IoU metric.
        Keyword arguments:
        - predicted (Tensor): Can be a (N, K, H, W) tensor of
        predicted scores obtained from the model for N examples and K classes,
        or (N, H, W) tensor of integer values between 0 and K-1.
        - target (Tensor): Can be a (N, K, H, W) tensor of
        target scores for N examples and K classes, or (N, H, W) tensor of
        integer values between 0 and K-1.
        """
        # Dimensions check
        assert predicted.size(0) == target.size(0), \
            'number of targets and predicted outputs do not match'
        assert predicted.dim() == 3 or predicted.dim() == 4, \
            "predictions must be of dimension (N, H, W) or (N, K, H, W)"
        assert target.dim() == 3 or target.dim() == 4, \
            "targets must be of dimension (N, H, W) or (N, K, H, W)"

        # If the tensor is in categorical format convert it to integer format
        if predicted.dim() == 4:
            _, predicted = predicted.max(1)
        if target.dim() == 4:
            _, target = target.max(1)

        self.conf_metric.add(predicted.view(-1), target.view(-1))

    def value(self):
        """Computes the IoU and mean IoU.
        The mean computation ignores NaN elements of the IoU array.
        Returns:
            Tuple: (IoU, mIoU). The first output is the per class IoU,
            for K classes it's numpy.ndarray with K elements. The second output,
            is the mean IoU.
        """
        conf_matrix = self.conf_metric.value()
        if self.ignore_index is not None:
            conf_matrix[:, self.ignore_index] = 0
            conf_matrix[self.ignore_index, :] = 0
        true_positive = np.diag(conf_matrix)
        false_positive = np.sum(conf_matrix, 0) - true_positive
        false_negative = np.sum(conf_matrix, 1) - true_positive

        # Just in case we get a division by 0, ignore/hide the error
        with np.errstate(divide='ignore', invalid='ignore'):
            iou = true_positive / (true_positive + false_positive + false_negative)

        return iou, np.nanmean(iou)

### Model specifications and training

In [ ]:
model = U_Net(n_channels, n_classes)
'''state=torch.load('/content/drive/MyDrive/PH2/model_wts/U_Net_weights_1.pth') # ekhane previous weight ta src korbi 
model.load_state_dict(state['model_state'])'''
model = model.to(device)
criterion = nn.MSELoss()
#DiceCoeff()
#iou = IoU(num_classes=2)
criterion = criterion.to(device)
optim = torch.optim.Adam(model.parameters(), l_rate)
'''optim.load_state_dict(state['optimizer_state'])'''

In [ ]:
# Training

def train(model, epoch_n, optim,criterion):
  best_loss=1000.0 
  train_loss_list = []
  val_loss_list = []
  for epoch in range(epoch_n):
    model.train()
    train_loss=miou=total=dice_score=0.0
    for _,data in enumerate(train_load):
      image = data['image'].to(device, dtype=torch.float32)
      mask_type = torch.float32 if n_classes == 1 else torch.long
      mask = data['mask'].to(device, dtype=mask_type)
      with torch.set_grad_enabled(True):
        mask_gen = model(image)
        loss = criterion(mask_gen,mask)
        loss.backward()
        optim.step()
      optim.zero_grad()
      train_loss += loss.item()
      total += mask.size(0)
      # iou.add(mask_gen, mask)
      # _ ,miou_temp =  iou.value()
      # miou += miou_temp
      #dice_score += dice_coeff(mask_gen,mask).item()
    val_loss, dice_score_val = eval(model, criterion)
    epoch_train_loss = train_loss/len(train_ds)
    print("Epoch: {}".format(epoch+1))
    print('-'*10)
    print('Train Loss: {:.4f}'.format(epoch_train_loss))
    epoch_val_loss = val_loss/len(val_ds)
    print('Val Loss: {:.4f}'.format(epoch_val_loss))
    print('\n')
    # iou_score = miou*100/total
    '''print('Dice score train: {:.4f}'.format(dice_score))
    print('\n')
    print('Dice score val: {:.4f}'.format(dice_score_val))
    print('\n')'''
    '''plt.imshow(mask_gen.cpu().detach().numpy()[-3][0])
    plt.imshow(mask.cpu().detach().numpy()[-1].transpose((1,2,0)))
    _, axarr = plt.subplots(1,3)
    print(image.cpu().detach().numpy()[-1].shape)
    axarr[2] = plt.imshow(mask_gen.cpu().detach().numpy()[-1][0])
    axarr[1] = plt.imshow(mask.cpu().detach().numpy()[-1][0].transpose((1,2,0)))
    axarr[0] = plt.imshow(image.cpu().detach().numpy()[-3].transpose((1,2,0)))
    input()'''
    if epoch == epoch_n:
      best_loss = epoch_val_loss
      best_model_wts = copy.deepcopy(model.state_dict())
      state={
          "model_state":model.state_dict(),
          "optimizer_state":optim.state_dict(),
            }
      torch.save(state,'/content/drive/MyDrive/PH2/model_wts/U_Net_weights_1.pth') 
    train_loss_list = train_loss_list + [epoch_train_loss]
    val_loss_list = val_loss_list + [epoch_val_loss]
  model = model.load_state_dict(best_model_wts)
  print("The model with the best performance has a score of :{:.4f}".format(best_loss))
  return model, train_loss_list, val_loss_list  


def eval(model, criterion):
  model.eval()
  with torch.no_grad():
    val_loss=dice_score=0.0
    for _,data in enumerate(val_load):
      image = data['image'].to(device, dtype=torch.float32)
      mask_type = torch.float32 if n_classes == 1 else torch.long
      mask = data['mask'].to(device, dtype=mask_type)
      mask_gen = model(image)
      loss = criterion(mask_gen, mask)
      val_loss += loss.item()
      dice_score += dice_coeff(mask_gen,mask).item()
  return val_loss, dice_score


In [ ]:
model, train_loss_list, val_loss_list = train(model, epoch_n, optim, criterion)

### Mask generation of entire dataset

In [ ]:
def eval(model):
  model.eval()
  with torch.no_grad():
    # val_loss=dice_score=0.0
    for i,data in enumerate(val_load):
      image = data['image'].to(device, dtype=torch.float32)
      mask_type = torch.float32 if n_classes == 1 else torch.long
      mask = data['mask'].to(device, dtype=mask_type)
      mask_gen = model(image)
      path = input_val[i]
      file_name = path[-10:-4]
      for element, class_t in class_list:
        if element == file_name:
          class_type = class_t
      if class_type == '0':
        plt.imsave(path[:-17]+'_classifier/common_nevus/'+path[-10:], mask_gen[0][0].cpu().detach().numpy())
      if class_type == '1':
        plt.imsave(path[:-17]+'_classifier/atypical_nevus/'+path[-10:], mask_gen[0][0].cpu().detach().numpy())
      if class_type == '2':
        plt.imsave(path[:-17]+'_classifier/melanoma/'+path[-10:], mask_gen[0][0].cpu().detach().numpy())
      # loss = criterion(mask_gen, mask)
      # val_loss += loss.item()
      # dice_score += dice_coeff(mask_gen,mask).item()

In [ ]:
eval(model)